In [1]:
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Splitter, BRepAlgoAPI_Fuse
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakePolygon, BRepBuilderAPI_MakeFace, BRepBuilderAPI_Transform, BRepBuilderAPI_MakePolygon, BRepBuilderAPI_MakeWire
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakePrism
from OCC.Core.gp import gp_Pnt, gp_Vec,gp_Trsf,gp_Ax1,gp_Dir, gp_Circ, gp_Ax2
from OCC.Core import TopAbs
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir, gp_Pln
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakePolygon, BRepBuilderAPI_MakeFace, BRepBuilderAPI_MakeEdge
from OCC.Core.BRep import BRep_Polygon3D
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.BOPAlgo import BOPAlgo_Splitter
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib_Add
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.GC import GC_MakeArcOfCircle
from OCC.Core.Geom import Geom_Curve, Geom_Line
from OCC.Display.SimpleGui import init_display
from OCC.Core.GC import GC_MakeArcOfCircle
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve

import math
import numpy as np

In [2]:
def create_prism(elevation_height, height, truss_height, upper_extension_length=0, lower_extension_length=0, angle_degrees=30, scale_factor=100):
    '''
    创建一个扶梯，其形状由输入的参数定义。
    Parameters:
    elevation_height: 抬升高度
    height: 梯级高度
    truss_height: 桁架高度
    upper_extension_length: 上长度
    lower_extension_length: 下长度
    angle_degrees: 扶梯的角度
    '''
    angle = math.radians(angle_degrees)
    sin_val = math.sin(angle)
    cos_val = math.cos(angle)
    tan_val = math.tan(angle)
    upper_length = upper_extension_length + 2566
    lower_length = lower_extension_length + 2199
    p1 = gp_Pnt(0, 0, (elevation_height+height)/scale_factor)
    p2 = gp_Pnt(upper_length / scale_factor, 0, (elevation_height + height) / scale_factor)
    p3 = gp_Pnt((upper_length + elevation_height / tan_val) / scale_factor, 0, height / scale_factor)
    p4 = gp_Pnt((upper_length + elevation_height / tan_val + lower_length) / scale_factor, 0, height / scale_factor)
    p5 = gp_Pnt((upper_length + elevation_height / tan_val + lower_length) / scale_factor, 0, 0)
    p6 = gp_Pnt((upper_length + elevation_height / tan_val - height*tan_val + ((height/cos_val)-truss_height)/sin_val) / scale_factor, 0, 0)
    p7 = gp_Pnt((upper_length - truss_height * sin_val + (height - (truss_height * cos_val)) / tan_val) / scale_factor, 0, elevation_height / scale_factor)
    p8 = gp_Pnt(0, 0, elevation_height / scale_factor)
 
    points = [p1, p2, p3, p4, p5, p6, p7, p8]
    # 创建多边形线框
    top_polygon = BRepBuilderAPI_MakePolygon()
    for pnt in points:
        top_polygon.Add(pnt)
    top_polygon.Close()

    # 创建面
    top_face = BRepBuilderAPI_MakeFace(top_polygon.Wire())
    # 沿Y轴向外推移2000单位以创建立体
    main_prism = BRepPrimAPI_MakePrism(top_face.Shape(), gp_Vec(0, 2000/scale_factor, 0)).Shape()

    # 定义圆
    circ = gp_Circ(gp_Ax2(gp_Pnt((1594.5 + upper_extension_length) / scale_factor, 0, (elevation_height + height - 2730) / scale_factor), gp_Dir(0, 1, 0)), 3000 / scale_factor)

    # 创建弧
    arc_of_circle = GC_MakeArcOfCircle(circ, 0, angle, True).Value()

    # 转换为边
    curve_edge = BRepBuilderAPI_MakeEdge(arc_of_circle).Edge()
    # 使用BRepAdaptor从边获取曲线
    adaptor1 = BRepAdaptor_Curve(curve_edge)
    first_param1, last_param1 = adaptor1.FirstParameter(), adaptor1.LastParameter()

    # 获取末端点的坐标
    start_point1 = adaptor1.Value(first_param1)
    end_point1 = adaptor1.Value(last_param1)
    normal_vector = gp_Dir(-1, 0, -1/tan_val)
    normal_line = Geom_Line(end_point1, normal_vector)
    perpendicular_edge = BRepBuilderAPI_MakeEdge(normal_line, 0, 150 / scale_factor).Edge()
    edge1 = BRepBuilderAPI_MakeEdge(Geom_Line(start_point1, gp_Dir(-1, 0, 0)), 0, 803.61 / scale_factor).Edge()
    adaptor2 = BRepAdaptor_Curve(edge1)
    last_param2 = adaptor2.LastParameter()
    end_point2 = adaptor2.Value(last_param2)
    edge2 = BRepBuilderAPI_MakeEdge(Geom_Line(end_point2, gp_Dir(0, 0, -1)), 0, 270 / scale_factor).Edge()
    adaptor3 = BRepAdaptor_Curve(edge2)
    last_param3 = adaptor3.LastParameter()
    end_point3 = adaptor3.Value(last_param3)
    adaptor4 = BRepAdaptor_Curve(perpendicular_edge)
    last_param4 = adaptor4.LastParameter()
    end_point4 = adaptor4.Value(last_param4)
    edge3 = BRepBuilderAPI_MakeEdge(end_point3, p2).Edge()
    edge4 = BRepBuilderAPI_MakeEdge(p2, end_point4).Edge()

    # 创建线框
    wire_builder = BRepBuilderAPI_MakeWire()

    # 添加所有边到线框
    wire_builder.Add(curve_edge)
    wire_builder.Add(perpendicular_edge)
    wire_builder.Add(edge1)
    wire_builder.Add(edge2)
    wire_builder.Add(edge3)
    wire_builder.Add(edge4)
    # 完成线框
    wire = wire_builder.Wire()
    # 创建面
    face = BRepBuilderAPI_MakeFace(wire, True)  # True 表示创建的面应该是平面的
    prism = BRepPrimAPI_MakePrism(face.Shape(), gp_Vec(0, 1980/scale_factor, 0)).Shape()
    
    # 定义下部圆
    lower_circ = gp_Circ(gp_Ax2(gp_Pnt((elevation_height / tan_val + 100.3 + upper_length) / scale_factor, 0, (1270 + height) / scale_factor), gp_Dir(0, 1, 0)), 1000 / scale_factor)

    # 创建弧
    lower_arc = GC_MakeArcOfCircle(lower_circ, math.pi, angle + math.pi, True).Value()

    # # 转换为边
    lower_curve_edge = BRepBuilderAPI_MakeEdge(lower_arc).Edge()
    # 使用BRepAdaptor从边获取曲线
    lower_adaptor1 = BRepAdaptor_Curve(lower_curve_edge)
    #获取末端点的参数位置
    lower_first_param1, lower_last_param1 = lower_adaptor1.FirstParameter(), lower_adaptor1.LastParameter()

    # # 获取末端点的坐标
    lower_start_point1 = lower_adaptor1.Value(lower_first_param1)
    lower_end_point1 = lower_adaptor1.Value(lower_last_param1)
    lower_edge1 = BRepBuilderAPI_MakeEdge(Geom_Line(lower_start_point1, gp_Dir(1, 0, 0)), 0, 1160.82 / scale_factor).Edge()
    lower_edge2 = BRepBuilderAPI_MakeEdge(Geom_Line(lower_end_point1, gp_Dir(-1, 0, tan_val)), 0, 304.14 / scale_factor).Edge()

    lower_adaptor2 = BRepAdaptor_Curve(lower_edge1)
    lower_last_param2 = lower_adaptor2.LastParameter()
    lower_end_point2 = lower_adaptor2.Value(lower_last_param2)
    lower_edge3 = BRepBuilderAPI_MakeEdge(Geom_Line(lower_end_point2, gp_Dir(0, 0, -1)), 0, 270 / scale_factor).Edge()

    lower_adaptor3 = BRepAdaptor_Curve(lower_edge2)
    lower_last_param3 = lower_adaptor3.LastParameter()
    lower_end_point3 = lower_adaptor3.Value(lower_last_param3)
    lower_edge4 = BRepBuilderAPI_MakeEdge(Geom_Line(lower_end_point3, gp_Dir(-1, 0, -1/tan_val)), 0, 150 / scale_factor).Edge()

    lower_adaptor4 = BRepAdaptor_Curve(lower_edge4)
    lower_last_param4 = lower_adaptor4.LastParameter()
    lower_end_point4 = lower_adaptor4.Value(lower_last_param4)
    lower_edge5 = BRepBuilderAPI_MakeEdge(lower_end_point4, p3).Edge()

    lower_adaptor5 = BRepAdaptor_Curve(lower_edge3)
    lower_last_param5 = lower_adaptor5.LastParameter()
    lower_end_point5 = lower_adaptor5.Value(lower_last_param5)
    lower_edge6 = BRepBuilderAPI_MakeEdge(lower_end_point5, p3).Edge()


    # 创建线框
    lower_wire_builder = BRepBuilderAPI_MakeWire()

    # 添加所有边到线框
    lower_wire_builder.Add(lower_curve_edge)
    lower_wire_builder.Add(lower_edge1)
    lower_wire_builder.Add(lower_edge2)
    lower_wire_builder.Add(lower_edge3)
    lower_wire_builder.Add(lower_edge4)
    lower_wire_builder.Add(lower_edge5)
    lower_wire_builder.Add(lower_edge6)

    # 完成线框
    lower_wire = lower_wire_builder.Wire()
    # 创建面
    lower_face = BRepBuilderAPI_MakeFace(lower_wire, True)  # True 表示创建的面应该是平面的
    lower_prism = BRepPrimAPI_MakePrism(lower_face.Shape(), gp_Vec(0, 1980/scale_factor, 0)).Shape()


    fuse1 = BRepAlgoAPI_Fuse(prism, main_prism).Shape()
    fused_shape = BRepAlgoAPI_Fuse(fuse1, lower_prism).Shape()
    
    return fused_shape
    

In [77]:
def calculate_split_points(elevation_height, height, upper_extension_length=0, angle_degrees=30, lower_slope_length=2025, scale_factor=100):
    '''
    Calculate a series of split points along a slope based on given parameters.

    Parameters:
        elevation_height (int): Elevation height.
        height (int): height.
        upper_extension_length (int): Extension length at the upper end.
        upper_slope_length (int): Length of the upper slope section.
        lower_slope_length (int): Length of the lower section.
        angle_degrees (int): Angle in degrees.
        scale_factor (int, optional): Scaling factor for coordinates.

    Returns:
        list of gp_Pnt: A list of split points.
    '''
    angle_radians = math.radians(angle_degrees)
    sin_val = math.sin(angle_radians)
    cos_val = math.cos(angle_radians)
    tan_val = math.tan(angle_radians)
    upper_length = upper_extension_length + 2566
    slope_legth = elevation_height / sin_val - lower_slope_length
    points = []
    current_distance = 0  # Start from the beginning of the slope

    # Calculate the starting point
    x_start = upper_length + elevation_height / tan_val - lower_slope_length * cos_val
    z_start = height + lower_slope_length * sin_val
    # Loop to generate points every 1200 units until reaching the lower length limit
    while current_distance <= slope_legth - 3630:
        x = x_start - current_distance * cos_val
        z = z_start + current_distance * sin_val
        points.append(gp_Pnt(x / scale_factor, 0, z / scale_factor))
        current_distance += 1200
    return points

In [4]:
def get_lower_coords(split_points, elevation_height, height, truss_height, upper_extension_length=0, lower_extension_length=0, angle_degrees=30, scale_factor=100):
    angle_radians = math.radians(angle_degrees)
    sin_val = math.sin(angle_radians)
    cos_val = math.cos(angle_radians)
    tan_val = math.tan(angle_radians)
    upper_length = upper_extension_length + 2566
    lower_length = lower_extension_length + 2199
    gp_p1 = split_points[0]
    p1 = [gp_p1.X(), gp_p1.Y(), gp_p1.Z()]
    p2 = [(upper_length + elevation_height / tan_val + lower_length) / scale_factor, 0, height / scale_factor]
    p3 = [(upper_length + elevation_height / tan_val + lower_length) / scale_factor, 0, 0]
    p4 = [p1[0] - (truss_height * sin_val) / scale_factor, p1[1], p1[2] - (truss_height * cos_val) / scale_factor]
    return [p1, p2, p3, p4]

def get_upper_coords(split_points, elevation_height, height, truss_height, upper_extension_length=0, lower_extension_length=0, angle_degrees=30, scale_factor=100):
    angle_radians = math.radians(angle_degrees)
    sin_val = math.sin(angle_radians)
    cos_val = math.cos(angle_radians)
    gp_p1 = split_points[0]
    p1 = [0, 0, (elevation_height+height)/scale_factor]
    p2 = [gp_p1.X(), gp_p1.Y(), gp_p1.Z()]
    p3 = [p2[0] - (truss_height * sin_val) / scale_factor, p2[1], p2[2] - (truss_height * cos_val) / scale_factor]
    p4 = [0, 0, elevation_height / scale_factor]
    return [p1, p2, p3, p4]


In [86]:
def get_split_idx(split_points, elevation_height, height, truss_height, upper_extension_length=0, lower_extension_length=0, angle_degrees=30, scale_factor=100):
    if angle_degrees == 30:
        if upper_extension_length == 0 and lower_extension_length == 0:
            if elevation_height <= 3470:
                return []
            elif elevation_height <= 6000:
                return [int(len(split_points) / 2)]
            else:
                split_idx = [3]
                mid = len(split_points) - 6
                while mid > 9:
                    split_idx.append(mid+2)
                    mid -= 9
                split_idx.append(int(len(split_points)-3))
                split_idx.sort()
                return split_idx
split_points = calculate_split_points(24000, 1270, 0, 30, 2025, 100)
print(len(split_points))
get_split_idx(split_points, 24000, 1270, 100, 0, 0, 30, 100)



36


[3, 14, 23, 32, 33]

In [55]:
scale_factor = 50
elevation_height = 6000
truss_height = 982
height = 1018
lower_slope_length = 2025
upper_extension_length = 0
lower_extension_length = 0
degree = 30

prism = create_prism(elevation_height, height, truss_height, upper_extension_length, lower_extension_length, degree, scale_factor)
#选择要分割的点的索引
split_idx = [0]
# 创建分割面
all_points = calculate_split_points(elevation_height, height, upper_extension_length, degree, lower_slope_length, scale_factor)
print(len(all_points))
split_points = [all_points[i] for i in split_idx]
direction = gp_Dir(-1/math.tan(math.radians(degree)), 0, 1)
planes = [gp_Pln(point, direction) for point in split_points]
faces = [BRepBuilderAPI_MakeFace(plane, -1000, 1000, -1000, 1000).Shape() for plane in planes]  # 创建足够大的面

# 初始化分割器并设置非破坏性操作
splitter = BOPAlgo_Splitter()
splitter.SetNonDestructive(True)
splitter.AddArgument(prism)  # 添加要被分割的立体

# 添加所有分割工具
for face in faces:
    splitter.AddTool(face)

# 执行分割
splitter.Perform()
result = splitter.Shape()
# 分离出所有子形状
exp = TopExp_Explorer()
exp.Init(result, TopAbs.TopAbs_SOLID)

sub_shapes = []

while exp.More():
    sub_shapes.append(exp.Current())
    exp.Next()

translation_vector = gp_Vec(20, 30, 40)
# 设定移动间隔
distance = 10
transformed_shapes = []

# 遍历形状列表，每个形状移动更远的距离
for i, shape in enumerate(sub_shapes):
    # 创建变换对象
    trsf = gp_Trsf()
    # 沿X轴移动，移动距离随着索引增加
    trsf.SetTranslation(gp_Vec(distance * i, 0, 0))
    
    # 应用变换
    transformer = BRepBuilderAPI_Transform(shape, trsf)
    # 获取变换后的形状
    transformed_shape = transformer.Shape()
    
    # 存储变换后的形状
    transformed_shapes.append(transformed_shape)

rnd = JupyterRenderer()
colors = ["#DB0570", "#0506DB", "#66CCFF", "#FFCC00", "#FF6600", "#FF0000", "#00FF00", "#0000FF", "#FF00FF"]
for s, c in zip(sub_shapes, colors):
    rnd.DisplayShape(s, render_edges=True, shape_color=c)
# print(sub_shapes[0].Coords())
rnd.Display()


7


TraitError: The 'rotation' trait of a GridHelper instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.

TraitError: The 'rotation' trait of a GridHelper instance contains an Enum of an Euler which expected any of ['XYZ', 'YZX', 'ZXY', 'XZY', 'YXZ', 'ZYX'], not the str 'xyz'.

In [6]:
from OCC.Core.BRep import BRep_Builder
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Common
from OCC.Core.gp import gp_Pnt

# Create two boxes
box1 = BRepPrimAPI_MakeBox(gp_Pnt(0, 0, 0), gp_Pnt(10, 10, 10)).Shape()
box2 = BRepPrimAPI_MakeBox(gp_Pnt(5, 5, 5), gp_Pnt(15, 15, 15)).Shape()

# Find the intersection using BRepAlgoAPI_Common
intersect = BRepAlgoAPI_Common(box1, box2)

# Check if the intersection is null or not
if intersect.IsDone() and not intersect.Shape().IsNull():
    print("Collision detected!")
else:
    print("No collision.")


Collision detected!


In [30]:
a = 10
b = []
while a > 9:
    a -= 9
    print(a)

1
